In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas

In [ ]:
! exa -l

# Thailand extract from Raghav

In [ ]:
df = pandas.read_csv('baci_trade_THA.csv', dtype={'product_code':'str'}) \
    .drop(columns=[
        't',
        'i',
        'j',
        'export_country_name',
        'import_country_name',
        'refining_stage',
        'export_continent',
        'import_continent'
    ]) \
    .rename(columns={'product_code': 'product_code_h5'})

df.export_country_code = df.export_country_code.fillna('TWN')  # Other Asia, n.e.s.
df.import_country_code = df.import_country_code.fillna('TWN')  # Other Asia, n.e.s.
df.trade_quantity_tons = df.trade_quantity_tons.fillna(0)  # code 271600 is electrical energy, recorded as NaN tons
df.product_code_h5 = df.product_code_h5.str.zfill(6)
df.shape, df.dropna().shape

In [ ]:
product_codes_hs_to_isic_with_desc = pandas.read_csv('JobID-64_Concordance_HS_to_I3.CSV', encoding='latin-1', dtype='str').rename(columns={
    'HS - Combined  Product Code': 'product_code_hs',
    'HS - Combined  Product Description': 'product_desc_hs',
    'ISIC Revision 3 Product Code': 'isic_code',
    'ISIC Revision 3 Product Description': 'isic_desc'
})
product_codes_hs_to_isic= product_codes_hs_to_isic_with_desc[['product_code_hs','isic_code']]

In [ ]:
product_codes_h5_to_h4_with_desc = pandas.read_csv('JobID-92_Concordance_H5_to_H4.CSV', encoding='latin-1', dtype='str').rename(columns={
    'HS 2017 Product Code': 'product_code_h5',
    'HS 2017 Product Description': 'product_desc_h5',
    'HS 2012 Product Code': 'product_code_h4',
    'HS 2012 Product Description': 'product_desc_h4'
})
product_codes_h5_to_h4 = product_codes_h5_to_h4_with_desc[['product_code_h5','product_code_h4']]

In [ ]:
product_codes_h4_to_isic = pandas.read_csv('JobID-81_Concordance_H4_to_I3.CSV', encoding='latin-1', dtype='str').rename(columns={
    'HS 2012 Product Code': 'product_code_h4',
    'HS 2012 Product Description': 'product_desc_h4',
    'ISIC Revision 3 Product Code': 'isic_code',
    'ISIC Revision 3 Product Description': 'isic_desc'
})[['product_code_h4','isic_code']]

In [ ]:
extra_codes_to_isic = pandas.read_csv('extra_concordance.csv', dtype='str')[['product_code','isic_code']]

In [ ]:
df = df \
    .merge(product_codes_h5_to_h4, on='product_code_h5', validate='many_to_one', how='left') \
    .merge(product_codes_h4_to_isic, on='product_code_h4', validate='many_to_one', how='left')
df.shape

In [ ]:
df1 = df[~df.isic_code.isna()].copy()
df2 = df[df.isic_code.isna()] \
    .drop(columns=['isic_code']) \
    .copy() \
    .merge(product_codes_hs_to_isic, left_on='product_code_h5', right_on='product_code_hs', validate='many_to_one', how='left')
df2_clean = df2[~df2.isic_code.isna()].copy()
df3 = df2[df2.isic_code.isna()] \
    .drop(columns=['isic_code']) \
    .copy() \
    .merge(extra_codes_to_isic, left_on='product_code_h5', right_on='product_code', validate='many_to_one', how='left')
df3_clean = df3[~df3.isic_code.isna()].copy()

In [ ]:
len(df1), len(df2), len(df3), len(df3[df3.isic_code.isna()])

In [ ]:
all = pandas.concat([df1, df2_clean, df3])

In [ ]:
isic_trade = all \
    [['isic_code', 'export_country_code', 'import_country_code', 'trade_value_thousandUSD', 'trade_quantity_tons']] \
    .groupby(['isic_code', 'export_country_code', 'import_country_code']) \
    .sum()

In [ ]:
isic_trade.trade_value_thousandUSD.sum(), all.trade_value_thousandUSD.sum(), df.trade_value_thousandUSD.sum()

In [ ]:
partner_trade = all \
    [['export_country_code', 'import_country_code', 'trade_value_thousandUSD', 'trade_quantity_tons']] \
    .groupby(['export_country_code', 'import_country_code']) \
    .sum()

In [ ]:
# major_partner_trade = partner_trade.query('trade_value_thousandUSD > 1e6')

tha_exports = partner_trade \
    .loc["THA"] \
    .rename(columns={
        "trade_value_thousandUSD": "THA_export_trade_value_thousandUSD",
        "trade_quantity_tons": "THA_export_trade_quantity_tons"
    })
tha_exports.index.name = 'country_code'
tha_exports

tha_imports = partner_trade.reset_index() \
    .query("import_country_code == 'THA'") \
    .set_index("export_country_code") \
    .drop(columns=['import_country_code']) \
    .rename(columns={
        "trade_value_thousandUSD": "THA_import_trade_value_thousandUSD",
        "trade_quantity_tons": "THA_import_trade_quantity_tons"
    })

tha_imports.index.name = 'country_code'
tha_trade_balance = tha_exports.join(tha_imports)

In [ ]:
tha_trade_balance.query('THA_export_trade_value_thousandUSD > 1e6')[['THA_import_trade_value_thousandUSD', 'THA_export_trade_value_thousandUSD']].plot(kind='bar')

plt.savefig('baci_major_trade_balance_tha.png')

In [ ]:
mat_df = partner_trade.query('trade_value_thousandUSD > 1e7').reset_index() \
    .pivot(index='export_country_code', columns='import_country_code', values='trade_value_thousandUSD')

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(mat_df)
cbar = ax.figure.colorbar(im, ax=ax, label='1e7 thousand USD')

ax.set_xticks(np.arange(len(mat_df.columns)), labels=mat_df.columns, fontsize='small')
ax.set_xlabel('Country importing')
ax.set_yticks(np.arange(len(mat_df.index)), labels=mat_df.index, fontsize='small')
ax.set_ylabel('Country exporting')
# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=90, ha="right",
         rotation_mode="anchor")
ax.set_title('Value of trade over 1bn USD, 2021')

plt.savefig('baci_major_trade_tha.png')

In [ ]:
isic_structure = pandas.read_csv('ISIC_Rev_3_english_structure.csv')
isic_structure

In [ ]:
sector_trade = isic_trade.reset_index() \
    .merge(isic_structure[['sector', 'code']], left_on='isic_code', right_on='code', how='left') \
    .drop(columns=['isic_code', 'code']) \
    .groupby(['sector','export_country_code','import_country_code']) \
    .sum()

In [ ]:
summary = sector_trade.reset_index()
summary['export'] = (summary.export_country_code == 'THA')
summary.drop(columns=['export_country_code','import_country_code']).groupby(['export','sector']).sum()

In [ ]:
sector_trade.to_csv("baci_sector_trade_THA.csv")

In [ ]:
! exa ~/projects/open-gira/results/input/capital-stocks/

In [ ]:
! exa /data/incoming/wenz-2023-dose-reported-subnational-output/